In [505]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [506]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphic

In [507]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [508]:
import warnings
warnings.filterwarnings(action = 'ignore')

In [509]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
chromeDriver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

In [510]:
column = ["시즌", "FA", "출생", "순","이름","팀","WAR","출장","타석","타수","득점","안타","2루타","3루타","홈런","루타","타점","도루","도루실패","볼넷","사구","고4","삼진","병살","희타","희비","타율","출루","장타","OPS","wOBA","wRC+","WAR*","WPA"]
df = pd.DataFrame(columns=column)

seasons = range(1982, 2023)

start_idx = 3

# chromeDriver.get("http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=2020&ye=2020&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&pa=0&si=&cn=&sn=400")

for season in seasons:
    chromeDriver.get("http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=0&ys={}&ye={}&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=400&si=&cn=".format(season, season))
    html = chromeDriver.page_source
    bsObject = BeautifulSoup(html,"html.parser")
    temp = bsObject.find_all("table")[1]
    templen = len(temp.find_all("tr"))

    for i in range(2, templen):
        tempTr = temp.find_all("tr")[i]
        if(tempTr.find("th") is not None):
            continue
        row = {}
        row[column[0]] = season
        column_idx = start_idx
        for j in range(31):            
            tempTd = tempTr.find_all("td")[j].text
            if(tempTd == "" and j > 0):
                continue
            if j == 1 :
                  row[column[2]] = tempTr.find_all("td")[1].find("a").get('href').split('birth=')[1][:4]
            if j == 2 :
                tempTd = tempTd + '_' + [each_element.split(':') for each_element in tempTr.find_all("td")[j].find_all('span')[1].get('style').split(';')[:-1]][2][1]

            row[column[column_idx]] = tempTd
            column_idx += 1
        df = df.append(row,ignore_index = True)
    tempTr.find_all("td")[2]
    tempTr.find_all("td")[2].find_all('span')
    tempTr.find_all("td")[2].find_all('span')[1].get('style')
    tempTr.find_all("td")[2].find_all('span')[1].get('style').split(';')[:-1]
    [each_element.split(':') for each_element in tempTr.find_all("td")[2].find_all('span')[1].get('style').split(';')[:-1]]
    [each_element.split(':') for each_element in tempTr.find_all("td")[2].find_all('span')[1].get('style').split(';')[:-1]][2]
    [each_element.split(':') for each_element in tempTr.find_all("td")[2].find_all('span')[1].get('style').split(';')[:-1]][2][1]

df['FA'] = 0
df

,시즌,FA,출생,순,이름,팀,WAR,출장,타석,타수,...,희타,희비,타율,출루,장타,OPS,wOBA,wRC+,WAR*,WPA
0,1982,0,1943,1,백인천,82MLF_#1245AB,5.88,72,298,250,...,0,3,.412,.497,.740,1.237,.528,227.0,5.88,NaN
1,1982,0,1949,2,윤동균,82OLF_#000042,4.13,77,334,284,...,0,4,.342,.428,.532,.960,.430,164.6,4.13,NaN
2,1982,0,1958,3,이만수,82삼C_#001EC9,4.12,78,322,270,...,0,1,.289,.399,.500,.899,.405,147.6,4.12,NaN
3,1982,0,1952,4,김봉연,82해DH_#DB0000,3.68,74,304,269,...,0,1,.331,.405,.636,1.040,.453,180.5,3.68,NaN
4,1982,0,1949,5,김우열,82OCF_#000042,3.46,62,255,210,...,0,1,.310,.428,.533,.961,.426,161.9,3.46,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10032,2022,0,2003,353,조세진,22롯RF_#041642,-0.76,39,88,86,...,1,0,.186,.195,.221,.416,.192,5.7,-0.76,-0.64
10033,2022,0,2000,354,유로결,22한LF_#ED4C00,-0.82,30,68,60,...,3,0,.133,.200,.133,.333,.164,-11.4,-0.82,-1.05
10034,2022,0,1999,355,정보근,22롯C_#041642,-0.91,95,226,199,...,10,1,.191,.250,.216,.466,.218,24.1,-0.91,-1.78
10035,2022,0,1984,356,박경수,22K2B_black,-1.01,100,194,166,...,2,1,.120,.234,.193,.427,.213,21.9,-1.01,-2.16


In [511]:
# pd.set_option("display.max_columns",32)

In [512]:
df["포지션"] = df["팀"].str.split("_").str[0]
df["포지션"] = df["포지션"].str.slice(3,)

In [513]:
df

,시즌,FA,출생,순,이름,팀,WAR,출장,타석,타수,...,희비,타율,출루,장타,OPS,wOBA,wRC+,WAR*,WPA,포지션
0,1982,0,1943,1,백인천,82MLF_#1245AB,5.88,72,298,250,...,3,.412,.497,.740,1.237,.528,227.0,5.88,NaN,LF
1,1982,0,1949,2,윤동균,82OLF_#000042,4.13,77,334,284,...,4,.342,.428,.532,.960,.430,164.6,4.13,NaN,LF
2,1982,0,1958,3,이만수,82삼C_#001EC9,4.12,78,322,270,...,1,.289,.399,.500,.899,.405,147.6,4.12,NaN,C
3,1982,0,1952,4,김봉연,82해DH_#DB0000,3.68,74,304,269,...,1,.331,.405,.636,1.040,.453,180.5,3.68,NaN,DH
4,1982,0,1949,5,김우열,82OCF_#000042,3.46,62,255,210,...,1,.310,.428,.533,.961,.426,161.9,3.46,NaN,CF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10032,2022,0,2003,353,조세진,22롯RF_#041642,-0.76,39,88,86,...,0,.186,.195,.221,.416,.192,5.7,-0.76,-0.64,RF
10033,2022,0,2000,354,유로결,22한LF_#ED4C00,-0.82,30,68,60,...,0,.133,.200,.133,.333,.164,-11.4,-0.82,-1.05,LF
10034,2022,0,1999,355,정보근,22롯C_#041642,-0.91,95,226,199,...,1,.191,.250,.216,.466,.218,24.1,-0.91,-1.78,C
10035,2022,0,1984,356,박경수,22K2B_black,-1.01,100,194,166,...,1,.120,.234,.193,.427,.213,21.9,-1.01,-2.16,2B


In [514]:
df.loc[df["팀"].str.contains('#C9'),"팀"] = "KIA"
df.loc[df["팀"].str.contains('#74'),"팀"] = "키움"
df.loc[df["팀"].str.contains('#000054'),"팀"] = "두산"
df.loc[df["팀"].str.contains('#002266'),"팀"] = "NC"
df.loc[df["팀"].str.contains('#001EC9'),"팀"] = "삼성"
df.loc[df["팀"].str.contains('#FF1212'),"팀"] = "SK"
df.loc[df["팀"].str.contains('#ED4C00'),"팀"] = "한화"
df.loc[df["팀"].str.contains('#041642'),"팀"] = "롯데"
df.loc[(df["팀"].str.contains('K') & df["팀"].str.contains('black')),"팀"] = "KT"
df.loc[(df["팀"].str.contains('L') & df["팀"].str.contains('black')),"팀"] = "LG"
df.loc[df["팀"].str.contains('#cf112d'),"팀"] = "SK"

In [515]:
df

,시즌,FA,출생,순,이름,팀,WAR,출장,타석,타수,...,희비,타율,출루,장타,OPS,wOBA,wRC+,WAR*,WPA,포지션
0,1982,0,1943,1,백인천,82MLF_#1245AB,5.88,72,298,250,...,3,.412,.497,.740,1.237,.528,227.0,5.88,NaN,LF
1,1982,0,1949,2,윤동균,82OLF_#000042,4.13,77,334,284,...,4,.342,.428,.532,.960,.430,164.6,4.13,NaN,LF
2,1982,0,1958,3,이만수,삼성,4.12,78,322,270,...,1,.289,.399,.500,.899,.405,147.6,4.12,NaN,C
3,1982,0,1952,4,김봉연,82해DH_#DB0000,3.68,74,304,269,...,1,.331,.405,.636,1.040,.453,180.5,3.68,NaN,DH
4,1982,0,1949,5,김우열,82OCF_#000042,3.46,62,255,210,...,1,.310,.428,.533,.961,.426,161.9,3.46,NaN,CF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10032,2022,0,2003,353,조세진,롯데,-0.76,39,88,86,...,0,.186,.195,.221,.416,.192,5.7,-0.76,-0.64,RF
10033,2022,0,2000,354,유로결,한화,-0.82,30,68,60,...,0,.133,.200,.133,.333,.164,-11.4,-0.82,-1.05,LF
10034,2022,0,1999,355,정보근,롯데,-0.91,95,226,199,...,1,.191,.250,.216,.466,.218,24.1,-0.91,-1.78,C
10035,2022,0,1984,356,박경수,KT,-1.01,100,194,166,...,1,.120,.234,.193,.427,.213,21.9,-1.01,-2.16,2B


In [516]:
df = df.reindex(columns=["순", "시즌", "FA", "출생", "이름","팀","포지션","WAR","출장","타석","타수","득점","안타","2루타","3루타","홈런","루타","타점","도루","도루실패","볼넷","사구","고4","삼진","병살","희타","희비","타율","출루","장타","OPS","wOBA","wRC+","WAR*","WPA"])

In [517]:
df

,순,시즌,FA,출생,이름,팀,포지션,WAR,출장,타석,...,희타,희비,타율,출루,장타,OPS,wOBA,wRC+,WAR*,WPA
0,1,1982,0,1943,백인천,82MLF_#1245AB,LF,5.88,72,298,...,0,3,.412,.497,.740,1.237,.528,227.0,5.88,NaN
1,2,1982,0,1949,윤동균,82OLF_#000042,LF,4.13,77,334,...,0,4,.342,.428,.532,.960,.430,164.6,4.13,NaN
2,3,1982,0,1958,이만수,삼성,C,4.12,78,322,...,0,1,.289,.399,.500,.899,.405,147.6,4.12,NaN
3,4,1982,0,1952,김봉연,82해DH_#DB0000,DH,3.68,74,304,...,0,1,.331,.405,.636,1.040,.453,180.5,3.68,NaN
4,5,1982,0,1949,김우열,82OCF_#000042,CF,3.46,62,255,...,0,1,.310,.428,.533,.961,.426,161.9,3.46,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10032,353,2022,0,2003,조세진,롯데,RF,-0.76,39,88,...,1,0,.186,.195,.221,.416,.192,5.7,-0.76,-0.64
10033,354,2022,0,2000,유로결,한화,LF,-0.82,30,68,...,3,0,.133,.200,.133,.333,.164,-11.4,-0.82,-1.05
10034,355,2022,0,1999,정보근,롯데,C,-0.91,95,226,...,10,1,.191,.250,.216,.466,.218,24.1,-0.91,-1.78
10035,356,2022,0,1984,박경수,KT,2B,-1.01,100,194,...,2,1,.120,.234,.193,.427,.213,21.9,-1.01,-2.16


In [518]:
df["팀"].value_counts()

롯데                1206
한화                1182
삼성                1160
LG                1081
SK                 751
                  ... 
92태LCF_#005000       1
92해1B_#DB0000        1
92쌍SS_#353535        1
92OCF_#000042        1
17k_black            1
Name: 팀, Length: 888, dtype: int64

In [519]:
df["포지션"].value_counts()

       1512
C      1286
P      1207
2B      861
RF      814
       ... 
KSS       1
L1B       1
태1B       1
빙CF       1
키1B       1
Name: 포지션, Length: 115, dtype: int64

In [520]:
p_data = df[df["포지션"] == "P"].index

In [521]:
df = df.drop(p_data)

In [522]:
n_data = df[(df["포지션"] == "")].index

In [523]:
df = df.drop(n_data)

In [524]:
df

,순,시즌,FA,출생,이름,팀,포지션,WAR,출장,타석,...,희타,희비,타율,출루,장타,OPS,wOBA,wRC+,WAR*,WPA
0,1,1982,0,1943,백인천,82MLF_#1245AB,LF,5.88,72,298,...,0,3,.412,.497,.740,1.237,.528,227.0,5.88,NaN
1,2,1982,0,1949,윤동균,82OLF_#000042,LF,4.13,77,334,...,0,4,.342,.428,.532,.960,.430,164.6,4.13,NaN
2,3,1982,0,1958,이만수,삼성,C,4.12,78,322,...,0,1,.289,.399,.500,.899,.405,147.6,4.12,NaN
3,4,1982,0,1952,김봉연,82해DH_#DB0000,DH,3.68,74,304,...,0,1,.331,.405,.636,1.040,.453,180.5,3.68,NaN
4,5,1982,0,1949,김우열,82OCF_#000042,CF,3.46,62,255,...,0,1,.310,.428,.533,.961,.426,161.9,3.46,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10032,353,2022,0,2003,조세진,롯데,RF,-0.76,39,88,...,1,0,.186,.195,.221,.416,.192,5.7,-0.76,-0.64
10033,354,2022,0,2000,유로결,한화,LF,-0.82,30,68,...,3,0,.133,.200,.133,.333,.164,-11.4,-0.82,-1.05
10034,355,2022,0,1999,정보근,롯데,C,-0.91,95,226,...,10,1,.191,.250,.216,.466,.218,24.1,-0.91,-1.78
10035,356,2022,0,1984,박경수,KT,2B,-1.01,100,194,...,2,1,.120,.234,.193,.427,.213,21.9,-1.01,-2.16


In [525]:
df["팀"].value_counts()

롯데               896
삼성               892
한화               857
LG               776
SK               552
                ... 
88OCF_#000042      1
96해SS_#DB0000      1
95태SS_#005000      1
95해2B_#DB0000      1
92해SS_#DB0000      1
Name: 팀, Length: 766, dtype: int64

In [526]:
df["포지션"].value_counts()

C      1286
2B      861
RF      814
3B      791
LF      786
       ... 
두CF       1
태RF       1
해2B       1
쌍1B       1
키1B       1
Name: 포지션, Length: 113, dtype: int64

In [527]:
df.loc[df["포지션"]=="K3B","포지션"] = "3B"
df.loc[df["포지션"]=="K2B","포지션"] = "2B"
df.loc[df["포지션"]=="SLF","포지션"] = "LF"
df.loc[df["포지션"]=="한CF","포지션"] = "CF"
df.loc[df["포지션"]=="SC","포지션"] = "C"
df.loc[df["포지션"]=="KC","포지션"] = "C"

In [528]:
df["포지션"].value_counts()

C      1293
2B      865
RF      814
3B      794
LF      790
       ... 
L1B       1
태RF       1
쌍1B       1
현2B       1
키1B       1
Name: 포지션, Length: 107, dtype: int64

In [529]:
df

,순,시즌,FA,출생,이름,팀,포지션,WAR,출장,타석,...,희타,희비,타율,출루,장타,OPS,wOBA,wRC+,WAR*,WPA
0,1,1982,0,1943,백인천,82MLF_#1245AB,LF,5.88,72,298,...,0,3,.412,.497,.740,1.237,.528,227.0,5.88,NaN
1,2,1982,0,1949,윤동균,82OLF_#000042,LF,4.13,77,334,...,0,4,.342,.428,.532,.960,.430,164.6,4.13,NaN
2,3,1982,0,1958,이만수,삼성,C,4.12,78,322,...,0,1,.289,.399,.500,.899,.405,147.6,4.12,NaN
3,4,1982,0,1952,김봉연,82해DH_#DB0000,DH,3.68,74,304,...,0,1,.331,.405,.636,1.040,.453,180.5,3.68,NaN
4,5,1982,0,1949,김우열,82OCF_#000042,CF,3.46,62,255,...,0,1,.310,.428,.533,.961,.426,161.9,3.46,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10032,353,2022,0,2003,조세진,롯데,RF,-0.76,39,88,...,1,0,.186,.195,.221,.416,.192,5.7,-0.76,-0.64
10033,354,2022,0,2000,유로결,한화,LF,-0.82,30,68,...,3,0,.133,.200,.133,.333,.164,-11.4,-0.82,-1.05
10034,355,2022,0,1999,정보근,롯데,C,-0.91,95,226,...,10,1,.191,.250,.216,.466,.218,24.1,-0.91,-1.78
10035,356,2022,0,1984,박경수,KT,2B,-1.01,100,194,...,2,1,.120,.234,.193,.427,.213,21.9,-1.01,-2.16


In [530]:
df.to_csv("/content/drive/MyDrive/KBO_anlaysis/batter.csv", header = True, index = True)

In [531]:
column_pitcher = ["시즌", "FA", "출생", "순","이름","팀","자책",'출장','완투','완봉','선발','승','패','세','홀드','이닝','실점','자책','타자','안타','2타','3타','홈런','볼넷','고4','사구','삼진','보크','폭투','ERA','FIP','WHIP','ERA+','FIP+','WAR','WPA']

column = column_pitcher

df = pd.DataFrame(columns=column)

seasons = range(1982, 2023)

start_idx = 3

for season in seasons:
    # chromeDriver.get("http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=0&ys={}&ye={}&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=400&si=&cn=".format(season, season))
    chromeDriver.get("http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys={}&ye={}&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn=".format(season, season))
    html = chromeDriver.page_source
    bsObject = BeautifulSoup(html,"html.parser")
    temp = bsObject.find_all("table")[1]
    templen = len(temp.find_all("tr"))
    for i in range(2, templen):
        tempTr = temp.find_all("tr")[i]
        if(tempTr.find("th") is not None):
            continue
        row = {}
        row[column[0]] = season
        column_idx = start_idx
        for j in range(33):
            tempTd = tempTr.find_all("td")[j].text
            if(tempTd == "" and j > 0):
                continue
            if j == 1 :
                row[column[2]] = tempTr.find_all("td")[1].find("a").get('href').split('birth=')[1][:4]
            if j == 2 :
                tempTd = tempTd + '_' + [each_element.split(':') for each_element in tempTr.find_all("td")[j].find_all('span')[1].get('style').split(';')[:-1]][2][1]

            row[column[column_idx]] = tempTd
            column_idx += 1
        df = df.append(row,ignore_index = True)
    tempTr.find_all("td")[2]
    tempTr.find_all("td")[2].find_all('span')
    tempTr.find_all("td")[2].find_all('span')[1].get('style')
    tempTr.find_all("td")[2].find_all('span')[1].get('style').split(';')[:-1]
    [each_element.split(':') for each_element in tempTr.find_all("td")[2].find_all('span')[1].get('style').split(';')[:-1]]
    [each_element.split(':') for each_element in tempTr.find_all("td")[2].find_all('span')[1].get('style').split(';')[:-1]][2]
    [each_element.split(':') for each_element in tempTr.find_all("td")[2].find_all('span')[1].get('style').split(';')[:-1]][2][1]

df['FA'] = 0
df

,시즌,FA,출생,순,이름,팀,자책,출장,완투,완봉,...,삼진,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,WAR,WPA
0,1982,0,1956,1,박철순,82O_#000042,46,36,15,2,...,4,1.84,2.82,0.97,211.2,137.8,9.54,NaN,NaN,NaN
1,1982,0,1953,2,황규봉,82삼_#001EC9,61,47,8,2,...,4,2.47,2.67,1.00,160.6,148.5,8.62,NaN,NaN,NaN
2,1982,0,1958,3,노상수,82롯_#041642,76,44,12,2,...,4,2.94,3.03,1.18,134.9,131.3,7.05,NaN,NaN,NaN
3,1982,0,1955,4,하기룡,82M_#1245AB,49,43,7,4,...,6,2.30,2.95,1.11,172.3,134.4,6.78,NaN,NaN,NaN
4,1982,0,1954,5,권영호,82삼_#001EC9,47,32,6,3,...,1,2.37,3.91,1.07,167.5,101.3,5.12,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,2022,0,1990,26,장민재,22한_#ED4C00,50,32,0,0,...,75,0,3,3.55,4.69,1.29,114.0,87.0,2.38,0.64
1226,2022,0,1999,27,곽빈,22두_#000054,62,27,0,0,...,138,1,9,3.78,4.04,1.38,105.3,98.3,2.36,0.45
1227,2022,0,1990,28,김재윤,22K_black,24,61,0,0,...,78,1,2,3.26,2.65,1.01,126.5,155.1,2.32,0.57
1228,2022,0,1999,29,정우영,22L_black,17,67,0,0,...,40,0,0,2.64,4.34,1.38,150.7,92.9,2.17,1.79


In [532]:
df.loc[df["팀"].str.contains('#C9'),"팀"] = "KIA"

In [533]:
df.loc[df["팀"].str.contains('#74'),"팀"] = "키움"

In [534]:
df.loc[df["팀"].str.contains('#000054'),"팀"] = "두산"

In [535]:
df.loc[df["팀"].str.contains('#002266'),"팀"] = "NC"

In [536]:
df.loc[df["팀"].str.contains('#001EC9'),"팀"] = "삼성"

In [537]:
df.loc[df["팀"].str.contains('#FF1212'),"팀"] = "SK"

In [538]:
df.loc[df["팀"].str.contains('#ED4C00'),"팀"] = "한화"

In [539]:
df.loc[df["팀"].str.contains('#cf112d'),"팀"] = "SK"

In [540]:
df.loc[df["팀"].str.contains('#041642'),"팀"] = "롯데"

In [541]:
df.loc[(df["팀"].str.contains('K') & df["팀"].str.contains('black')),"팀"] = "KT"

In [542]:
df.loc[(df["팀"].str.contains('L') & df["팀"].str.contains('black')),"팀"] = "LG"

In [543]:
df.loc[df["팀"].str.contains('#cf112d'),"팀"] = "SK"

In [548]:
df

,시즌,FA,출생,순,이름,팀,자책,출장,완투,완봉,...,삼진,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,WAR,WPA
0,1982,0,1956,1,박철순,82O_#000042,46,36,15,2,...,4,1.84,2.82,0.97,211.2,137.8,9.54,NaN,NaN,NaN
1,1982,0,1953,2,황규봉,삼성,61,47,8,2,...,4,2.47,2.67,1.00,160.6,148.5,8.62,NaN,NaN,NaN
2,1982,0,1958,3,노상수,롯데,76,44,12,2,...,4,2.94,3.03,1.18,134.9,131.3,7.05,NaN,NaN,NaN
3,1982,0,1955,4,하기룡,82M_#1245AB,49,43,7,4,...,6,2.30,2.95,1.11,172.3,134.4,6.78,NaN,NaN,NaN
4,1982,0,1954,5,권영호,삼성,47,32,6,3,...,1,2.37,3.91,1.07,167.5,101.3,5.12,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,2022,0,1990,26,장민재,한화,50,32,0,0,...,75,0,3,3.55,4.69,1.29,114.0,87.0,2.38,0.64
1226,2022,0,1999,27,곽빈,두산,62,27,0,0,...,138,1,9,3.78,4.04,1.38,105.3,98.3,2.36,0.45
1227,2022,0,1990,28,김재윤,KT,24,61,0,0,...,78,1,2,3.26,2.65,1.01,126.5,155.1,2.32,0.57
1228,2022,0,1999,29,정우영,LG,17,67,0,0,...,40,0,0,2.64,4.34,1.38,150.7,92.9,2.17,1.79


In [550]:
df = df.drop(df.columns[[6]], axis=1)
df
df = df.reindex(columns=["순", "시즌", "FA", "출생", "이름","팀",'출장','완투','완봉','선발','승','패','세','홀드','이닝','실점','자책','타자','안타','2타','3타','홈런','볼넷','고4','사구','삼진','보크','폭투','ERA','FIP','WHIP','ERA+','FIP+','WAR','WPA'])
df

,순,시즌,FA,출생,이름,팀,출장,완투,완봉,선발,...,삼진,보크,폭투,ERA,FIP,WHIP,ERA+,FIP+,WAR,WPA
0,1,1982,0,1956,박철순,82O_#000042,NaN,15,2,19,...,4,1.84,2.82,0.97,211.2,137.8,9.54,NaN,NaN,NaN
1,2,1982,0,1953,황규봉,삼성,NaN,8,2,16,...,4,2.47,2.67,1.00,160.6,148.5,8.62,NaN,NaN,NaN
2,3,1982,0,1958,노상수,롯데,NaN,12,2,20,...,4,2.94,3.03,1.18,134.9,131.3,7.05,NaN,NaN,NaN
3,4,1982,0,1955,하기룡,82M_#1245AB,NaN,7,4,16,...,6,2.30,2.95,1.11,172.3,134.4,6.78,NaN,NaN,NaN
4,5,1982,0,1954,권영호,삼성,NaN,6,3,21,...,1,2.37,3.91,1.07,167.5,101.3,5.12,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,26,2022,0,1990,장민재,한화,NaN,0,0,25,...,75,0,3,3.55,4.69,1.29,114.0,87.0,2.38,0.64
1226,27,2022,0,1999,곽빈,두산,NaN,0,0,27,...,138,1,9,3.78,4.04,1.38,105.3,98.3,2.36,0.45
1227,28,2022,0,1990,김재윤,KT,NaN,0,0,0,...,78,1,2,3.26,2.65,1.01,126.5,155.1,2.32,0.57
1228,29,2022,0,1999,정우영,LG,NaN,0,0,0,...,40,0,0,2.64,4.34,1.38,150.7,92.9,2.17,1.79


In [551]:
df.to_csv("/content/drive/MyDrive/KBO_anlaysis/pitcher.csv", header = True, index = True)